In [1]:
from IPython.core.display import display, HTML
# Full display of notebook (width of screen)
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import basic libraries
import os, glob, sys
import numpy as np   
import pandas as pd 

# Define librearies needed for analysis
import matplotlib.pyplot as plt # Matplotlibrary
import seaborn as sns # Seaborn Library

from pylab import rcParams
rcParams['figure.figsize'] = 8, 8

feat_path = os.path.join(os.path.abspath(os.path.dirname(os.path.dirname(os.getcwd()))))
if feat_path not in sys.path:
    sys.path.append(feat_path)
    
%load_ext autoreload
%autoreload 2

np.random.seed(42)

# get additional tools
from src.utils import utils 

# Define path where data is located and folder to access to it
path_data = os.path.join(os.path.abspath(os.path.dirname(os.path.dirname(os.getcwd()))),'data','external')

In [2]:
# load data in pd dataframe
train = pd.read_csv(os.path.join(path_data,'train.csv'))
test = pd.read_csv(os.path.join(path_data,'test.csv'))

# Modify
to_drop = ['PassengerId','SibSp','Parch', 'Ticket', 'Cabin' ,'Name']

train = utils.pipeline_features(train, to_drop)
test = utils.pipeline_features(test, to_drop)

In [3]:
train.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,FamilySize,Title,Deck
0,0,3,0,1,0,0,2,2,7
1,1,1,1,2,3,1,2,3,2
2,1,3,1,1,1,0,1,1,7
3,1,1,1,2,3,0,2,3,2
4,0,3,0,2,1,0,1,2,7


In [4]:
test.head()

,Pclass,Sex,Age,Fare,Embarked,FamilySize,Title,Deck
0,3,0,2,0,2,1,2,7
1,3,1,2,0,0,2,3,7
2,2,0,3,1,2,1,2,7
3,3,0,1,1,0,1,2,7
4,3,1,1,1,0,3,3,7


In [8]:
#pip list